In [3]:
from mozo import merge_with_features, from_ln_pln_trans_to_predict, num_to_range_categories, save_model, stratified_sample

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

import math

pd.set_option('display.max_colwidth', None)


from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error

from scikitplot.estimators import plot_learning_curve

import time
from datetime import datetime
from threading import Timer



In [4]:
import pandas as pd
train = pd.read_hdf('~/pml7/module3/input/train.car_price.h5')
test = pd.read_hdf('~/pml7/module3/input/test.car_price.h5') #zbiór testowy bez odpowiedzi

In [5]:
train, test = merge_with_features('ft_price_test.csv', train, test)


train
Index(['car_id', 'pln_to_eur', 'price_value_pln',
       'price_value_pln_log_transl'],
      dtype='object')
test
Index(['car_id', 'pln_to_eur'], dtype='object')


In [6]:
train, test = merge_with_features('ft_new_param_test.csv', train, test)
train, test = merge_with_features('ft_new_param_2_encoded_train.csv', train, test)


train
Index(['car_id', 'new_param_rok-produkcji',
       'new_param_rok_produkcji_minus_aktualny_rok',
       'new_param_pierwsza-rejestracja-date', 'new_param_przebieg',
       'new_param_moc', 'new_param_pojemność-skokowa',
       'new_param_liczba-drzwi', 'new_param_liczba-miejsc',
       'new_param_serwisowany-w-aso', 'new_param_pierwszy-właściciel',
       'new_param_bezwypadkowy', 'new_param_zarejestrowany-w-polsce',
       'new_param_uszkodzony', 'new_param_leasing-concession',
       'new_param_możliwość-finansowania', 'new_param_filtr-cząstek-stałych',
       'new_param_metalik', 'new_param_perłowy',
       'new_param_akryl-(niemetalizowany)',
       'new_param_pierwsza_rejestracja_ten_rok',
       'new_param_pierwsza_rejestracja_poprzedni_rok',
       'new_param_pierwsza_rejestracja_2_lata',
       'new_param_pierwsza_rejestracja_3_lata',
       'new_param_pierwsza_rejestracja_4_5_lat',
       'new_param_pierwsza_rejestracja_6_7_lat',
       'new_param_pierwsza_rejestracja_8_

In [7]:
train, test = merge_with_features('ft_new_param_3_encoded_train.csv', train, test)


train
Index(['car_id', 'encoded_seller_type', 'encoded_seller_name',
       'encoded_seller_address', 'encoded_new_param_marka_pojazdu',
       'new_param_marka-pojazdu', 'encoded_new_param_model_pojazdu',
       'new_param_model', 'encoded_new_param_wersja_pojazdu',
       'new_param_wersja', 'encoded_new_param_kod_silnika',
       'new_param_kod-silnika'],
      dtype='object')
test
Index(['car_id', 'encoded_seller_type', 'encoded_seller_name',
       'encoded_seller_address', 'encoded_new_param_marka_pojazdu',
       'new_param_marka-pojazdu', 'encoded_new_param_model_pojazdu',
       'new_param_model', 'encoded_new_param_wersja_pojazdu',
       'new_param_wersja', 'encoded_new_param_kod_silnika',
       'new_param_kod-silnika'],
      dtype='object')


# Stratyfikacja:
- marka
- rocznik
- osob/ciez
- cena pln log
- netto, vat, negocjacja
- wypadkowy
- nowy/uzywany
- seller_type

In [ ]:
train[['seller_address', 'seller_name', 'seller_type']]

In [8]:
train['encoded_price_currency'] = train.apply(lambda row: 1 if row['price_currency'] == 'PLN' else 2, axis=1)
import re
train['netto'] = train.apply(lambda row: 1 if re.search('Netto', row['price_details']) else 0, axis=1)
train['faktura_VAT'] = train.apply(lambda row: 1 if re.search('Faktura VAT', row['price_details']) else 0, axis=1)
train['cena_do_negocjacji'] = train.apply(lambda row: 1 if re.search('negocjacji', row['price_details']) else 0, axis=1)

In [9]:
total = pd.concat([train, test], ignore_index=True)

In [10]:
num_to_range_categories(train, 'price_value_pln', n_cat=10)

perc_60     11021
perc_40     10940
perc_80     10746
perc_90     10684
perc_20     10666
perc_00     10574
perc_10     10567
perc_70     10529
perc_50     10434
perc_30     10332
perc_100        1
Name: range_price_value_pln_cat, dtype: int64
['perc_00', 'perc_10', 'perc_100', 'perc_20', 'perc_30', 'perc_40', 'perc_50', 'perc_60', 'perc_70', 'perc_80', 'perc_90']
0


In [11]:
num_to_range_categories(total, 'new_param_rok-produkcji', n_cat=4) #20
total['encoded_range_new_param_rok_produkcji_cat'] = total.apply(lambda row: int(row['range_new_param_rok_produkcji_cat'].replace('perc_', '')), axis=1)
print(total['encoded_range_new_param_rok_produkcji_cat'].value_counts() )

perc_25     42058
perc_50     41877
perc_75     40044
perc_00     33095
perc_100     3542
Name: range_new_param_rok_produkcji_cat, dtype: int64
['perc_00', 'perc_100', 'perc_25', 'perc_50', 'perc_75']
0
25     42058
50     41877
75     40044
0      33095
100     3542
Name: encoded_range_new_param_rok_produkcji_cat, dtype: int64


In [12]:
num_to_range_categories(total, 'new_param_przebieg', n_cat=4) #30
total['encoded_range_new_param_przebieg_cat'] = total.apply(lambda row: int(row['range_new_param_przebieg_cat'].replace('perc_', '')), axis=1)
print(total['encoded_range_new_param_przebieg_cat'].value_counts() )

perc_75     40380
perc_50     40235
perc_00     40151
perc_25     39849
perc_100        1
Name: range_new_param_przebieg_cat, dtype: int64
['perc_00', 'perc_100', 'perc_25', 'perc_50', 'perc_75']
0
75     40380
50     40235
0      40151
25     39849
100        1
Name: encoded_range_new_param_przebieg_cat, dtype: int64


In [13]:
num_to_range_categories(total, 'new_param_moc', n_cat=4) #15
total['encoded_range_new_param_moc_cat'] = total.apply(lambda row: int(row['range_new_param_moc_cat'].replace('perc_', '')), axis=1)
print(total['encoded_range_new_param_moc_cat'].value_counts() )

perc_50     48058
perc_75     40372
perc_25     36164
perc_00     36021
perc_100        1
Name: range_new_param_moc_cat, dtype: int64
['perc_00', 'perc_100', 'perc_25', 'perc_50', 'perc_75']
0
50     48058
75     40372
25     36164
0      36021
100        1
Name: encoded_range_new_param_moc_cat, dtype: int64


In [14]:
num_to_range_categories(total, 'new_param_pojemność-skokowa', n_cat=4) #10
total['encoded_range_new_param_pojemność_skokowa_cat'] = total.apply(lambda row: int(row['range_new_param_pojemność_skokowa_cat'].replace('perc_', '')), axis=1)
print(total['encoded_range_new_param_pojemność_skokowa_cat'].value_counts() )

perc_75     44167
perc_50     40405
perc_00     38229
perc_25     37814
perc_100        1
Name: range_new_param_pojemność_skokowa_cat, dtype: int64
['perc_00', 'perc_100', 'perc_25', 'perc_50', 'perc_75']
0
75     44167
50     40405
0      38229
25     37814
100        1
Name: encoded_range_new_param_pojemność_skokowa_cat, dtype: int64


In [51]:
filter_col = [ 'car_id', 
 'range_new_param_rok_produkcji_cat', 'encoded_range_new_param_rok_produkcji_cat', 
    'range_new_param_przebieg_cat',  'encoded_range_new_param_przebieg_cat', 
    'range_new_param_moc_cat',  'encoded_range_new_param_moc_cat', 
    'range_new_param_pojemność_skokowa_cat', 'encoded_range_new_param_pojemność_skokowa_cat', 
]

In [52]:
total.loc[~total['price_value'].isnull(), ].columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'range_new_param_rok-produkcji_cat', 'range_new_param_przebieg_cat',
       'encoded_range_new_param_przebieg_cat', 'range_new_param_moc_cat',
       'encoded_range_new_param_moc_cat',
       'range_new_param_pojemność-skokowa_cat',
       'range_new_param_pojemność_skokowa_cat',
       'encoded_range_new_param_pojemność_skokowa_cat',
       'range_new_param_rok_produkcji_cat',
       'encoded_range_new_param_rok_produkcji_cat'],
      dtype='object', length=268)

In [53]:
total.loc[~total['price_value'].isnull(), filter_col].to_csv('output/ft_range_param_encoded_train.csv', index=False) 
total.loc[total['price_value'].isnull(), filter_col].to_csv('output/ft_range_param_encoded_test.csv', index=False) 


In [144]:
math.ceil((len(train.index)) * 30 / 100)

31949

In [145]:
math.ceil((len(test.index)) * 100 / 100)

54122

In [15]:
train, test = merge_with_features('ft_range_param_encoded_test.csv', train, test)


train
Index(['car_id', 'range_new_param_rok_produkcji_cat',
       'encoded_range_new_param_rok_produkcji_cat',
       'range_new_param_przebieg_cat', 'encoded_range_new_param_przebieg_cat',
       'range_new_param_moc_cat', 'encoded_range_new_param_moc_cat',
       'range_new_param_pojemność_skokowa_cat',
       'encoded_range_new_param_pojemność_skokowa_cat'],
      dtype='object')
test
Index(['car_id', 'range_new_param_rok_produkcji_cat',
       'encoded_range_new_param_rok_produkcji_cat',
       'range_new_param_przebieg_cat', 'encoded_range_new_param_przebieg_cat',
       'range_new_param_moc_cat', 'encoded_range_new_param_moc_cat',
       'range_new_param_pojemność_skokowa_cat',
       'encoded_range_new_param_pojemność_skokowa_cat'],
      dtype='object')


In [25]:
strat_features = [
    'range_price_value_pln_cat', 'faktura_VAT', 'cena_do_negocjacji',# 'netto', 'encoded_price_currency',
    #'encoded_seller_address',
    'range_new_param_rok_produkcji_cat', 
    'range_new_param_przebieg_cat', 
    #'range_new_param_moc_cat', 'range_new_param_pojemność_skokowa_cat',  
             ###'new_param_rok-produkcji', 'new_param_przebieg', 'new_param_moc', 'new_param_pojemność-skokowa', 
    #'new_param_liczba-drzwi', 'new_param_liczba-miejsc',
    #'new_param_bezwypadkowy', 'new_param_serwisowany-w-aso', 'new_param_zarejestrowany-w-polsce', 'new_param_pierwszy-właściciel', 'new_param_możliwość-finansowania',
    #'new_param_metalik',
    #'new_param_kraj_pochodzenia_2', 
    #'new_param_typ', 'new_param_rodzaj_paliwa_2', 'new_param_skrzynia_biegów_2', 'new_param_napęd_2', 
    #   'encoded_new_param_model_pojazdu',
    #   'encoded_new_param_marka_pojazdu',
    #'new_param_kolor_2', 

]

for cl in strat_features:
    cl_new = cl.replace('-', '_')
    train.rename({cl: cl_new}, axis=1, inplace=True)
    print(train[cl_new].value_counts())
    print('==============================================================================')


perc_60     11021
perc_40     10940
perc_80     10746
perc_90     10684
perc_20     10666
perc_00     10574
perc_10     10567
perc_70     10529
perc_50     10434
perc_30     10332
perc_100        1
Name: range_price_value_pln_cat, dtype: int64
0    80052
1    26442
Name: faktura_VAT, dtype: int64
1    59980
0    46514
Name: cena_do_negocjacji, dtype: int64
perc_95     11765
perc_35      7496
perc_45      7312
perc_30      6942
perc_50      6353
perc_25      6204
perc_55      5932
perc_75      5781
perc_60      5462
perc_20      5234
perc_00      5045
perc_70      5013
perc_65      4940
perc_85      4763
perc_05      4310
perc_80      4304
perc_15      4050
perc_10      3244
perc_100     2344
Name: range_new_param_rok_produkcji_cat, dtype: int64
perc_07     5848
perc_03     4860
perc_73     4059
perc_50     3775
perc_57     3766
perc_77     3756
perc_63     3735
perc_87     3701
perc_37     3665
perc_97     3642
perc_40     3581
perc_47     3577
perc_27     3575
perc_20     3568
perc_17

In [19]:
train

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,...,cena_do_negocjacji,range_price_value_pln_cat,range_new_param_rok_produkcji_cat,encoded_range_new_param_rok_produkcji_cat,range_new_param_przebieg_cat,encoded_range_new_param_przebieg_cat,range_new_param_moc_cat,encoded_range_new_param_moc_cat,range_new_param_pojemność_skokowa_cat,encoded_range_new_param_pojemność_skokowa_cat
0,"[Osobowe, Dacia, Logan, Dacia Logan II (2012-)]","13:23, 5 marca 2018",PLN,Cena Brutto,45100.0,"Krasne 7A - 36-007 Krasne, rzeszowski, Podkarpackie (Polska)",AUTO SPEKTRUM,Dealer,False,False,...,0,perc_70,perc_100,100,perc_03,3,perc_13,13,perc_00,0
1,"[Osobowe, Peugeot, 3008, Peugeot 3008 I (2009-2016)]","15:48, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",28000.0,"Tomaszów Mazowiecki, tomaszowski, Łódzkie",None,Osoba prywatna,False,False,...,1,perc_50,perc_60,60,perc_73,73,perc_33,33,perc_20,20
2,"[Osobowe, Porsche, Cayenne, Porsche Cayenne II (2010-)]","06:30, 4 marca 2018",PLN,"Cena Netto, Do negocjacji, Faktura VAT",229500.0,"Kępińska 24a - 63-640 Bralin, kępiński, Wielkopolskie (Polska)",P.W. GRIB,Dealer,True,True,...,1,perc_90,perc_80,80,perc_17,17,perc_93,93,perc_90,90
3,"[Osobowe, Peugeot, 308, Peugeot 308 T7 (2008-2013)]","08:46, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",19500.0,"Limanowa, limanowski, Małopolskie",None,Osoba prywatna,False,False,...,1,perc_30,perc_50,50,perc_40,40,perc_27,27,perc_20,20
4,"[Osobowe, Volkswagen, Golf, Volkswagen Golf VII (2012-)]","20:00, 3 marca 2018",PLN,"Cena Brutto, Faktura VAT",164900.0,"Niciarniana 51/53 - 92-320 Łódź, Śródmieście (Polska)",KROTOSKI-CICHY Niciarniana 51/53 oraz Dąbrowskiego 216 ŁÓDŹ SAMOCHODY UŻYWANE,Dealer,True,True,...,0,perc_90,perc_95,95,perc_17,17,perc_93,93,perc_60,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106489,"[Osobowe, Hyundai, i20, Hyundai i20 I (2008-2014)]","15:12, 27 lutego 2018",PLN,"Cena Netto, Faktura VAT",17900.0,"ul.Gliwicka 102 - 42-600 Tarnowskie Góry, tarnogórski, Śląskie (Polska)",Auto Salon DZIUK Dealer Krajowych Samochodów Używanych.,Dealer,False,False,...,0,perc_30,perc_70,70,perc_27,27,perc_07,7,perc_00,0
106490,"[Osobowe, Škoda, RAPID]","16:23, 22 lutego 2018",PLN,"Cena Netto, Faktura VAT",22700.0,"Modlińska 253b - 03-120 Warszawa, Mazowieckie (Polska)",TIV-AUTO Modlińska253b SALON BEZWYPADKOWYCH WYSELEKCJONOWANYCH AUT Z POLSKICH SALONÓW,Dealer,False,False,...,0,perc_40,perc_70,70,perc_40,40,perc_07,7,perc_00,0
106491,"[Osobowe, Opel, Zafira, Opel Zafira B (2005-2011)]","21:12, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",15500.0,"Rzeszów, Podkarpackie",None,Osoba prywatna,False,True,...,1,perc_20,perc_30,30,perc_93,93,perc_40,40,perc_50,50
106492,"[Osobowe, Volkswagen, Passat, Volkswagen Passat B5 FL (2000-2005)]","16:10, 28 lutego 2018",PLN,"Cena Brutto, Do negocjacji",6490.0,"Katowicka 112 - 41-705 Ruda Śląska, Śląskie (Polska)",Auto Komis,Dealer,True,False,...,1,perc_00,perc_15,15,perc_93,93,perc_73,73,perc_90,90


In [20]:
train = train.sample(frac=1).reset_index(drop=True)

In [21]:
train

,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,...,cena_do_negocjacji,range_price_value_pln_cat,range_new_param_rok_produkcji_cat,encoded_range_new_param_rok_produkcji_cat,range_new_param_przebieg_cat,encoded_range_new_param_przebieg_cat,range_new_param_moc_cat,encoded_range_new_param_moc_cat,range_new_param_pojemność_skokowa_cat,encoded_range_new_param_pojemność_skokowa_cat
0,"[Osobowe, Škoda, Octavia, Škoda Octavia III (2013-)]","10:58, 4 marca 2018",PLN,"Cena Brutto, Faktura VAT",94599.0,"Aleje Jerozolimskie - 41-500 Warszawa, Śródmieście (Polska)",SuperAuto24.com-OD RĘKI.,Dealer,True,False,...,0,perc_80,perc_95,95,perc_03,3,perc_73,73,perc_10,10
1,"[Osobowe, Toyota, Verso]","18:57, 1 marca 2018",PLN,Cena Brutto,46400.0,"Wojska Polskiego 19c - 23-300 Janów Lubelski, janowski, Lubelskie (Polska)",Japancars,Dealer,True,True,...,0,perc_70,perc_60,60,perc_33,33,perc_67,67,perc_40,40
2,"[Osobowe, Volvo, XC 70]","00:01, 6 marca 2018",PLN,"Cena Brutto, Do negocjacji",13900.0,"Garwolińska 82 - 08-110 Siedlce, Mazowieckie (Polska)",F.H.U HUBERTO,Dealer,False,False,...,1,perc_20,perc_05,5,perc_63,63,perc_87,87,perc_80,80
3,"[Osobowe, BMW, Seria 5, 525, BMW Seria 5 E60 (2003-2010)]","11:42, 3 marca 2018",PLN,Cena Brutto,16999.0,"boczki 2 95-035 - 95-035 Boczki, zgierski, Łódzkie (Polska)",ARECAR,Dealer,False,False,...,0,perc_30,perc_35,35,perc_93,93,perc_80,80,perc_90,90
4,"[Osobowe, Renault, Scenic, Renault Scenic II (2003 - 2009)]","09:18, 4 marca 2018",PLN,Cena Brutto,10500.0,"Jagiellońska 88B - 03-215 Warszawa, Praga-Północ (Polska)",Autlet Sp. z o.o. - setki dobrze używanych aut w hali na terenie FSO w Warszawie.,Dealer,False,False,...,0,perc_10,perc_25,25,perc_80,80,perc_53,53,perc_70,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106489,"[Osobowe, Alfa Romeo, 147]","21:24, 28 lutego 2018",PLN,Cena Brutto,5900.0,"Wolska 3 / róg Wielickiej - 30-663 Kraków, Bieżanów-Prokocim (Polska)",ZYBI-CARS,Dealer,False,False,...,0,perc_00,perc_15,15,perc_57,57,perc_40,40,perc_30,30
106490,"[Osobowe, Volkswagen, Golf, Volkswagen Golf VII (2012-)]","10:31, 5 marca 2018",PLN,"Cena Brutto, Faktura VAT",99900.0,"Niciarniana 51/53 - 92-320 Łódź, Łódzkie (Polska)",Grupa KROTOSKI-CICHY Łódź,Dealer,True,True,...,0,perc_80,perc_95,95,perc_03,3,perc_73,73,perc_10,10
106491,"[Osobowe, Volkswagen, Passat, Volkswagen Passat B6 (2005-2010)]","14:18, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",19900.0,"Świdnica, świdnicki, Dolnośląskie",None,Osoba prywatna,True,False,...,1,perc_40,perc_30,30,perc_70,70,perc_60,60,perc_80,80
106492,"[Osobowe, Smart, Forfour, Smart Forfour II (2014-)]","15:45, 5 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",53600.0,"ul.Olsztyńska 1 - 51-423 Wrocław, Psie Pole (Polska)",Smart Center Wrocław,Dealer,False,True,...,1,perc_70,perc_85,85,perc_13,13,perc_13,13,perc_00,0


In [ ]:
train_30 = stratified_sample(train, strata=strat_features, size=31949, seed=None, keep_index= False)
train_30

/home/m.mozolewski_3736/pml7/konkurs/mozo.py:310: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['size'] = 1


In [ ]:
train_30.info()

In [ ]:
train_30['car_id'].to_csv('output/split_train_30.csv', index=False) 


In [86]:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):

    #rename cols
    strata2 = []
    
    for cl in strata:                             #df.columns[1:]:
        cl_new = cl.replace('-', '_')
        df.rename({cl: cl_new}, axis=1, inplace=True)
        strata2.append(cl_new)
        
        if df[cl_new].dtype == np.float64:
            df[cl_new] = df.apply(lambda row: int(row[cl_new]), axis=1)
            print(df[cl_new].value_counts())

    strata = strata2
    
    population = len(df)
    size = __smpl_size(population, size) #???
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df

def __smpl_size(population, size):
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

#stratified_sample(train, strata=['new_param_rok-produkcji'], size=21299, seed=None, keep_index= False)

/home/conda/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,breadcrumb,created_at,price_currency,price_details,price_value,seller_address,seller_name,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,...,pca_35,pca_36,pca_37,pca_38,pca_39,encoded_price_currency,netto,faktura_VAT,cena_do_negocjacji,price_value_range
0,"[Osobowe, Buick, Roadmaster]","10:22, 2 marca 2018",PLN,"Cena Brutto, Do negocjacji, Faktura VAT",95000.0,"Zakopiańska 167 - 30-435 Kraków, Podgórze (Polska)",Dealer Hyundai Inter Car Nowak Sp. z o.o.,Dealer,False,False,...,-0.009210,0.024342,0.132549,0.023451,0.015695,1,0,1,1,perc_88
1,"[Osobowe, Jaguar, Inny]","11:20, 1 marca 2018",EUR,Cena Brutto,110000.0,"Leszczyńska 6 - 43-300 Bielsko-Biała, Śląskie (Polska)",Classic-Group,Dealer,False,False,...,-0.009210,0.024342,0.132549,0.023451,0.015695,2,0,0,0,perc_90
2,"[Osobowe, Gaz, 67]","09:21, 5 marca 2018",EUR,Cena Netto,12000.0,"Mukaczewo, mukaczewo, Użhorod",None,Osoba prywatna,False,False,...,-0.009210,0.024342,0.132549,0.023451,0.015695,2,1,0,0,perc_21
3,"[Osobowe, Mercedes-Benz, Inny]","23:44, 2 marca 2018",PLN,Cena Brutto,19500.0,"Mława, mławski, Mazowieckie",None,Osoba prywatna,False,False,...,-0.009210,0.024342,0.132549,0.023451,0.015695,1,0,0,0,perc_39
4,"[Osobowe, Cadillac, Deville]","13:12, 3 marca 2018",PLN,"Cena Brutto, Do negocjacji",74900.0,"Amerykańskie Pojazdy Zabytkowe - 66-100 Sulechów, zielonogórski, Lubuskie (Polska)",usClassic24,Dealer,False,False,...,-0.009210,0.024342,0.132549,0.023451,0.015695,1,0,0,1,perc_83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21293,"[Osobowe, Volkswagen, T-Roc]","10:06, 4 marca 2018",PLN,"Cena Brutto, Faktura VAT",94500.0,"Lubowidzka 42 - 80-174 Gdańsk, Jasień (Polska)",PLICHTA,Dealer,True,True,...,0.402842,0.183590,-0.088018,1.228423,-0.229413,1,0,1,0,perc_87
21294,"[Osobowe, Audi, Q7, Audi Q7 II (2015-)]","18:33, 5 marca 2018",PLN,"Cena Brutto, Faktura VAT",247900.0,"ul. Sabały 58 - 02-174 Warszawa, Włochy (Polska)",CarConnection,Dealer,False,False,...,-0.788303,-0.828337,0.947864,-0.433512,-0.412924,1,0,1,0,perc_97
21295,"[Osobowe, Audi, A6, Audi A6 C7 (2011-)]","11:49, 4 marca 2018",PLN,"Cena Brutto, Faktura VAT",220890.0,"Al. Katowicka 43 - 05-830 Wolica, pruszkowski, Mazowieckie (Polska)",Audi Krotoski-Cichy,Dealer,True,False,...,1.822381,-0.291518,-0.193912,0.117652,1.292908,1,0,1,0,perc_97
21296,"[Osobowe, Volkswagen, Passat, Volkswagen Passat B8 (2014-)]","15:59, 5 marca 2018",PLN,"Cena Brutto, Faktura VAT",207060.0,"ul.Rzgowska 142/146 - 93-311 Łódź, Łódzkie (Polska)",Zimny Auto Sp.z o.o.,Dealer,True,False,...,1.078101,-0.886100,-0.526660,0.049760,-0.325036,1,0,1,0,perc_96
